In [68]:
import * as utils from '../_utils/mod.ts'

In [69]:
const input = await utils.loadMatrix({ day: 3, part: 2 })


In [70]:
import { assert } from 'https://deno.land/std@0.208.0/assert/assert.ts'

interface Location {
  row: number
  start: number
  end: number
  value: number
  gears: `${number}-${number}`[]
}

function findLocations(input: string[][]): Location[] {
  let rowIndex = 0
  const locations: Location[] = []
  for (const row of input) {
    let columnIndex = 0
    let start: number | null = null
    let rawNum = ''
    for (const column of row) {
      if (start === null) {
        if (parseInt(column)) {
          start = columnIndex
          rawNum += column
        } else {
          // Nothing to do, we are between numbers
        }
      } else {
        if (!Number.isInteger(parseInt(column))) {
          locations.push({
            start,
            end: columnIndex - 1,
            value: parseInt(rawNum),
            row: rowIndex,
            gears: [],
          })
          start = null
          rawNum = ''
        } else {
          // We are in a number so add a digit to the raw number
          rawNum += column
        }
      }
      columnIndex++
    }
    rowIndex++
  }
  return locations
}

const locations = findLocations(input)


In [71]:
import { assert } from 'https://deno.land/std@0.208.0/assert/assert.ts'

type Gear = `${number}-${number}`
interface Location {
  row: number
  start: number
  end: number
  value: number
  gears: Gear[]
}

function findGears(input: string[][], locations: Location[]): Gear[] {
  // Attached gear to location so we can check it later
  const gears: `${number}-${number}`[] = []
  for (const location of locations) {
    const { row, start, end } = location
    // Row above
    {
      const rowAbove = [...input[row - 1].slice(start - 1, end + 2)]
      let elementIndex = 0
      for (const element of rowAbove) {
        if (element === '*') {
          const gear = `${row - 1}-${start - 1 + elementIndex}` as Gear
          if (!gears.includes(gear)) {
            gears.push(gear)
          }
          location.gears.push(gear)
        }
        elementIndex++
      }
    }
    // Row below
    {
      const rowBelow = [...input[row + 1].slice(start - 1, end + 2)]
      let elementIndex = 0
      for (const element of rowBelow) {
        if (element === '*') {
          const gear = `${row + 1}-${start - 1 + elementIndex}` as Gear
          if (!gears.includes(gear)) {
            gears.push(gear)
          }
          location.gears.push(gear)
        }
        elementIndex++
      }
    }
    // Left of start
    {
      const element = input[row][start - 1]
      if (element === '*') {
        const gear = `${row}-${start - 1}` as Gear
        if (!gears.includes(gear)) {
          gears.push(gear)
        }
        location.gears.push(gear)
      }
    }
    // Right of end
    {
      const element = input[row][end + 1]
      if (element === '*') {
        const gear = `${row}-${end + 1}` as Gear
        if (!gears.includes(gear)) {
          gears.push(gear)
        }
        location.gears.push(gear)
      }
    }
  }
  return gears
}

const gears = findGears(input, locations)


In [72]:
type Gear = `${number}-${number}`
interface Location {
  row: number
  start: number
  end: number
  value: number
  gears: Gear[]
}

function findLocationsForGear(gear: Gear, locations: Location[]): [Location, Location] | null {
  const [first, second, ...rest] = locations.filter((location) => location.gears.includes(gear))
  if (first && second && rest.length === 0) {
    return [first, second]
  }
  return null
}

function calculateSumOfGears(gears: Gear[], locations: Location[]): number {
  let sum = 0
  for (const gear of gears) {
    const locationsForGear = findLocationsForGear(gear, locations)
    if (locationsForGear) {
      const [first, second] = locationsForGear
      sum += first.value * second.value
    }
  }
  return sum
}

In [73]:
console.log(calculateSumOfGears(gears, locations))

80253814
